# Jina 

Cloud-Native Neural Search Framework for Any Kind of Data.

Demo : https://master-jina-dleunji.endpoint.ainize.ai/

## Installation

In [ ]:
!pip install jina==2.1.13

In [ ]:
!pip install transformers==4.9.1

In [ ]:
pip install yolov5==5.0.7

In [ ]:
pip install lmdb==1.2.1

In [ ]:
pip install jina-commons@git+https://github.com/jina-ai/jina-commons.git#egg=jina-commons

In [ ]:
# clean up
! rm -rf workspace images query

**Downloading and unzipping data**

In [ ]:
! wget https://open-images.s3.eu-central-1.amazonaws.com/data.zip
! unzip data.zip

## **Executors**
In this section, we will start developing the necessary executors, for both query and index flows.

### **CLIPImageEncoder**
This encoder encodes an image into embeddings using the CLIP model. 
We want an executor that loads the CLIP model and encodes it during the query and index flows. 

Our executor should:
* support both **GPU** and **CPU**: That's why we will provision the `device` parameter and use it when encoding.
* be able to process documents in batches in order to use our resources effectively: To do so, we will use the parameter `batch_size`
* be able to encode the full image during the query flow and encode only chunks during the index flow: This can be achieved with `traversal_paths` and method `DocumentArray.batch`.

In [ ]:
from typing import Optional, Tuple

import torch
from jina import DocumentArray, Executor, requests
from jina.logging.logger import JinaLogger
from transformers import CLIPFeatureExtractor, CLIPModel


class CLIPImageEncoder(Executor):
    """Encode image into embeddings using the CLIP model."""

    def __init__(
        self,
        pretrained_model_name_or_path: str = "openai/clip-vit-base-patch32",
        device: str = "cpu",
        batch_size: int = 32,
        traversal_paths: Tuple = ("r",),
        *args,
        **kwargs,
    ):
        super().__init__(*args, **kwargs)
        self.batch_size = batch_size
        self.traversal_paths = traversal_paths
        self.pretrained_model_name_or_path = pretrained_model_name_or_path

        self.device = device
        self.preprocessor = CLIPFeatureExtractor.from_pretrained(
            pretrained_model_name_or_path
        )
        self.model = CLIPModel.from_pretrained(self.pretrained_model_name_or_path)
        self.model.to(self.device).eval()

    @requests
    def encode(self, docs: Optional[DocumentArray], parameters: dict, **kwargs):
        if docs is None:
            return

        traversal_paths = parameters.get("traversal_paths", self.traversal_paths)
        batch_size = parameters.get("batch_size", self.batch_size)
        document_batches_generator = docs.batch(
            traversal_paths=traversal_paths,
            batch_size=batch_size,
            require_attr="blob",
        )

        with torch.inference_mode():
            for batch_docs in document_batches_generator:
                blob_batch = [d.blob for d in batch_docs]
                tensor = self._generate_input_features(blob_batch)


                embeddings = self.model.get_image_features(**tensor)
                embeddings = embeddings.cpu().numpy()

                for doc, embed in zip(batch_docs, embeddings):
                    doc.embedding = embed

    def _generate_input_features(self, images):
        input_tokens = self.preprocessor(
            images=images,
            return_tensors="pt",
        )
        input_tokens = {
            k: v.to(torch.device(self.device)) for k, v in input_tokens.items()
        }
        return input_tokens

### **YoloV5Segmenter**
Since we want to retrieve small images in bigger images, the technique that we will havily rely on is segmenting. Basicly, we want to do object detection on the indexed images. This will generate bounding boxes around objects detected in side the images. The detected objects will be extracted and added as chunks to the original documents.
BTW, guess what is the state-of-the-art object detection model ?
Right, we will use YoloV5.


Our **YoloV5Segmenter** should be able to load the `ultralytics/yolov5` model from Torch hub, otherwise, load a custom model. To achieve this, the executor accepts parameter `model_name_or_path` which will be used when loading. We will implement the method `load` which checks if the model exists in the the Torch Hub, otherwise, loads it as a custom model.

For our use case, we will just rely on `yolov5s` (small version of `yolov5`). Of course, for better quality, you can choose a more complicated model or your custom model.

Furtheremore, we want **YoloV5Segmenter** to support both **GPU** and **CPU** and it should be able to process in batches. Again, this is as simple as adding parameters `device` and `batch_size` and using them during segmenting.

To perform segmenting, we will implement method `_segment_docs` which performs the following steps:
1. For each batch (a batch consists of several images), use the model to get predictions for each image
2. Each prediction of an image can contain several detections (because yolov5 will extract as much bounding boxes as possible, along with their confidence scores). We will filter out detections whose scores are below the `confidence_threshold` to keep good quality.

Each detection is actually 2 points -top left (x1, y1) and bottom right (x2, y2)- a confidence score and a class. We want be using the class of the detection, but it can be useful in other search applications.

3. With the detections that we have, we create crops (using the 2 points returned). Finally, we add these crops to image documents as chunks.

In [ ]:
from typing import Dict, Iterable, Optional

import torch
from jina import Document, DocumentArray, Executor, requests
from jina.logging.logger import JinaLogger
from jina_commons.batching import get_docs_batch_generator


class YoloV5Segmenter(Executor):

    def __init__(
        self,
        model_name_or_path: str = 'yolov5s',
        confidence_threshold: float = 0.3,
        batch_size: int = 32,
        device: str = 'cuda',
        *args,
        **kwargs
    ):
        super().__init__(*args, **kwargs)
        self.model_name_or_path = model_name_or_path
        self.confidence_threshold = confidence_threshold
        self.batch_size = batch_size

        if device != 'cpu' and not device.startswith('cuda'):
            self.logger.error('Torch device not supported. Must be cpu or cuda!')
            raise RuntimeError('Torch device not supported. Must be cpu or cuda!')
        if device == 'cuda' and not torch.cuda.is_available():
            self.logger.warning(
                'You tried to use GPU but torch did not detect your'
                'GPU correctly. Defaulting to CPU. Check your CUDA installation!'
            )
            device = 'cpu'
        self.device = torch.device(device)
        self.model = self._load(self.model_name_or_path)

    @requests
    def segment(
        self, docs: Optional[DocumentArray] = None, parameters: Dict = {}, **kwargs
    ):

        if docs:
            document_batches_generator = get_docs_batch_generator(
                docs,
                traversal_path=['r'],
                batch_size=parameters.get('batch_size', self.batch_size),
                needs_attr='blob',
            )
            self._segment_docs(document_batches_generator, parameters=parameters)

    def _segment_docs(self, document_batches_generator: Iterable, parameters: Dict):
        with torch.no_grad():
            for document_batch in document_batches_generator:
                images = [d.blob for d in document_batch]
                predictions = self.model(
                    images,
                    size=640,
                    augment=False,
                ).pred

                for doc, prediction in zip(document_batch, predictions):
                    for det in prediction:
                        x1, y1, x2, y2, conf, cls = det
                        if conf < parameters.get(
                            'confidence_threshold', self.confidence_threshold
                        ):
                            continue
                        crop = doc.blob[int(y1) : int(y2), int(x1) : int(x2), :]
                        doc.chunks.append(Document(blob=crop))

    def _load(self, model_name_or_path):
        if model_name_or_path in torch.hub.list('ultralytics/yolov5'):
            return torch.hub.load(
                'ultralytics/yolov5', model_name_or_path, device=self.device
            )
        else:
            return torch.hub.load(
                'ultralytics/yolov5', 'custom', model_name_or_path, device=self.device
            )


**Indexers**
After developing the encoder, we will need 2 kinds of indexers: 
1. SimpleIndexer: This indexer will take care of storing chunks of images. It also supports vector similarity search which is important match small query images against segments of original images.

2. LMDBStorage: LMDB is a simple memory-mapped transactional key-value store. It is convenient for this example because we can use it to store original indexed images so that we can retrieve them later. We will use it to create LMDBStorage which offers 2 functionnalities: indexing documents and retrieving documents by ID.

**SimpleIndexer**
To implement SimpleIndexer, we can leverage jina's `DocumentArrayMemmap`. You can read about this data type [here](https://docs.jina.ai/fundamentals/document/documentarraymemmap-api/).

Our indexer will create an instance of `DocumentArrayMemmap` when it's initialized. We want to store indexed documents inside the workspace folder that's why we pass the `workspace` attribute of the executor to `DocumentArrayMemmap`.

To index, we implement the method `index` which is bound to the index flow. It's as simple as extending the received docs to `DocumentArrayMemmap` instance.

On the other hand, for search, we implement the method `search`. We bind it to the query flow using the decorator `@requests(on='/search')`.
In jina, searching for query documents can be done by adding the results to the `matches` attribute of each query document. Since docs is a `DocumentArray` we can use method `match` to match query against the indexed documents.
Read more about `match` [here](https://docs.jina.ai/fundamentals/document/documentarray-api/#matching-documentarray-to-another).
There's another detail here. We already indexed documents before search, but we need to match query documents against chunks of the indexed images. Luckily, `DocumentArray.match` allows us to specify the traversal paths of right-hand-side parameter with parameter `traversal_rdarray`. Since we want to match the left side docs (query) against the chunks of the right side docs (indexed docs), we can specify that `traversal_rdarray=['c']`.

In [ ]:
from copy import deepcopy
from typing import Dict, Optional
import inspect

from jina import DocumentArray, Executor, requests
from jina.logging.logger import JinaLogger
from jina.types.arrays.memmap import DocumentArrayMemmap


class SimpleIndexer(Executor):
    def __init__(self, **kwargs):
        super().__init__(**kwargs)

        self._storage = DocumentArrayMemmap(
            self.workspace, key_length=kwargs.get('key_length', 64)
        )

    @requests(on='/index')
    def index(
        self,
        docs: Optional['DocumentArray'] = None,
        **kwargs,
    ):
        if docs:
            self._storage.extend(docs)

    @requests(on='/search')
    def search(
        self,
        docs: Optional['DocumentArray'] = None,
        parameters: Optional[Dict] = None,
        **kwargs,
    ):
        if not docs:
            return

        docs.match(self._storage, traversal_rdarray=['c'])


**LMDBStorage**

In order to implement the LMDBStorage, we need the following parts:

**I. Handler**

This will be a context manager that we will use when we access our LMDB database. We will create it as a standalone class.


**II. LMDBStorage constructor**

The constructor should initialize a few attributes:
* the `map_size` of the database
* the `default_traversal_paths`. Actually we need traversal paths because we will not be traversing documents in the same way during index and query flows. During index, we want to store the root documents. However, during query, we need to get the matches of documents by ID.
* the index file: again, to keep things clean, we will store the index file inside the workspace folder. Therefore we can use the `workspace` attribute.


**III. `LMDBStorage.index`**

In order to index documents, we first start a transaction (so that our Storage executor is ACID-compliant). Then, we traverse them according to the `traversal_paths` (will be root during). Finally, each document is serialized to string and then added to the database (the key is the document ID)


**IV. `LMDBStorage.search`**

Unlike search in the SimpleIndexer, we only which to get the matched Documents by ID and return them. Actually, the matched documents will be empty and will only contain the IDs. The goal is to return full matched documents using IDs.
To accomplish this, again, we start a transaction, traverse the matched documents, get each matched document by ID and use the results to fill our documents.

In [ ]:
import os
from typing import Dict, List

import lmdb
from jina import Document, DocumentArray, Executor, requests
from jina_commons import get_logger
from jina_commons.indexers.dump import export_dump_streaming, import_metas


class _LMDBHandler:
    def __init__(self, file, map_size):
        # see https://lmdb.readthedocs.io/en/release/#environment-class for usage
        self.file = file
        self.map_size = map_size

    @property
    def env(self):
        return self._env

    def __enter__(self):
        self._env = lmdb.Environment(
            self.file,
            map_size=self.map_size,
            subdir=False,
            readonly=False,
            metasync=True,
            sync=True,
            map_async=False,
            mode=493,
            create=True,
            readahead=True,
            writemap=False,
            meminit=True,
            max_readers=126,
            max_dbs=0,  # means only one db
            max_spare_txns=1,
            lock=True,
        )
        return self._env

    def __exit__(self, exc_type, exc_val, exc_tb):
        if hasattr(self, '_env'):
            self._env.close()


class LMDBStorage(Executor):
    def __init__(
        self,
        map_size: int = 1048576000,  # in bytes, 1000 MB
        default_traversal_paths: List[str] = ['r'],
        *args,
        **kwargs,
    ):
        super().__init__(*args, **kwargs)
        self.map_size = map_size
        self.default_traversal_paths = default_traversal_paths
        self.file = os.path.join(self.workspace, 'db.lmdb')
        if not os.path.exists(self.workspace):
            os.makedirs(self.workspace)

    def _handler(self):
        return _LMDBHandler(self.file, self.map_size)

    @requests(on='/index')
    def index(self, docs: DocumentArray, parameters: Dict, **kwargs):
        traversal_paths = parameters.get(
            'traversal_paths', self.default_traversal_paths
        )
        if docs is None:
            return
        with self._handler() as env:
            with env.begin(write=True) as transaction:
                for d in docs.traverse_flat(traversal_paths):
                    transaction.put(d.id.encode(), d.SerializeToString())

    @requests(on='/search')
    def search(self, docs: DocumentArray, parameters: Dict, **kwargs):
        traversal_paths = parameters.get(
            'traversal_paths', self.default_traversal_paths
        )
        if docs is None:
            return
        docs_to_get = docs.traverse_flat(traversal_paths)
        with self._handler() as env:
            with env.begin(write=True) as transaction:
                for i, d in enumerate(docs_to_get):
                    id = d.id
                    serialized_doc = Document(transaction.get(d.id.encode()))
                    d.update(serialized_doc)
                    d.id = id


**SimpleRanker**

You might think why do we need a ranker at all ?
Actually, a ranker is needed because we will matching small query images against chunks of parent documents. But how can we get back to parent documents (aka full images) given the chunks ? And what if 2 chunks belonging to the same parent are matched ?
We can solve this by aggregating the similarity scores of chunks that belong to the same parent (using an aggregation method, in our case, will be the `min` value).
So, for each query document, we perform the following:

1. We create an empty collection of parent scores. This collection will store, for each parent, a list of scores of its chunk documents.
2. For each match, since it's a originally a chunk document, we can retrieve its `parent_id`. And it's also a match document so we get its match score and add that value to the parent socres collection.
3. After processing all matches, we need to aggregate the scores of each parent using the `min` metric.
4. Finally, using the aggregated score values of parents, we can create a new list of matches (this time consisting of parents, not chunks). We also need to sort the matches list by aggregated scores.

When query documents exit the SimpleRanker, they now have matches consisting of parent documents. However, parent documents just have IDs. That why, during the previous steps, we created LMDBStorage so that we can actually retrieve parent documents by IDs and fill them with data.

In [ ]:
from collections import defaultdict
from typing import Dict, Iterable, Optional

from jina import Document, DocumentArray, Executor, requests


class SimpleRanker(Executor):
    def __init__(
        self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.metric = 'cosine'

    @requests(on='/search')
    def rank(
        self, docs: Optional[DocumentArray] = None, parameters: Dict = {}, **kwargs
    ):
        if docs is None:
            return

        for doc in docs:
            parents_scores = defaultdict(list)
            for m in DocumentArray([doc]).traverse_flat(['m']):
                parents_scores[m.parent_id].append(m.scores[self.metric].value)

            # Aggregate match scores for parent document and
            # create doc's match based on parent document of matched chunks
            new_matches = []
            for match_parent_id, scores in parents_scores.items():
                score = min(scores)

                new_matches.append(
                    Document(id=match_parent_id, scores={self.metric: score})
                )

            # Sort the matches
            doc.matches = new_matches
            doc.matches.sort(key=lambda d: d.scores[self.metric].value)


### Enabling GPU
Usually, indexing takes some time because YoloV5Segmenter and CLIPImageEncoder can be slow. However, you can speed up indexing by enabling GPU. To do so, you'll need to enable GPUs for the notebook:

* Navigate to Edit→Notebook Settings
* Select GPU from the Hardware Accelerator drop-down
* Change the device to 'cuda' in the following line

In [ ]:
device = 'cuda'

## Indexing
Now, after creating executors, it's time to use them in order to build an index Flow and index our data.


### Building the index Flow
We create a Flow object and add executors one after the other with the right parameters:

1. YoloV5Segmenter: We should also specify the device
2. CLIPImageEncoder: It also receives the device parameter. And since we only encode the chunks, we specify  `'traversal_paths': ['c']`
3. SimpleIndexer: We need to specify the workspace parameter
4. LMDBStorage: We also need to specify the workspace parameter. Furtheremore, the executor can be in parallel to the other branch. We can achieve this using `needs='gateway'`. Finally, we set `default_traversal_paths` to `['r']`
5. A final executor which just waits for both branchs.

After building the index Flow, we can plot it to verify that we're using the correct architecture.

In [ ]:
from jina import Flow
index_flow = Flow().add(uses=YoloV5Segmenter, name='segmenter', uses_with={'device': device}) \
  .add(uses=CLIPImageEncoder, name='encoder', uses_with={'device': device, 'traversal_paths': ['c']}) \
  .add(uses=SimpleIndexer, name='chunks_indexer', workspace='workspace') \
  .add(uses=LMDBStorage, name='root_indexer', workspace='workspace', needs='gateway', uses_with={'default_traversal_paths': ['r']}) \
  .add(name='wait_both', needs=['root_indexer', 'chunks_indexer'])
index_flow.plot()

Now it's time to index the dataset that we have downloaded. Actually, we will index images inside the `images` folder.
This helper function will convert image files into Jina Documents and yield them:

In [ ]:
from glob import glob
from jina import Document

def input_generator():
    for filename in glob('images/*.jpg'):
        doc = Document(uri=filename, tags={'filename': filename})
        doc.convert_image_uri_to_blob()
        yield doc

The final step in this section is to send the input documents to the index Flow. Note that indexing can take a while

In [ ]:
with index_flow:
    input_docs = input_generator()
    index_flow.post(on='/index', inputs=input_docs, show_progress=True)

## Searching:
Now, let's build the search Flow and use it in order to find sample query images.

Our Flow contains the following executors:

1. CLIPImageEncoder: It receives the device parameter. This time, since we want to encode root query documents, we specify that `'traversal_paths': ['r']`
2. SimpleIndexer: We need to specify the workspace parameter
3. SimpleRanker
4. LMDBStorage: First we specify the workspace parameter. Then we need to use a different traversal paths. This time we will be traversing matches: `'default_traversal_paths': ['m']`

In [ ]:
from jina import Flow
query_flow = Flow().add(uses=CLIPImageEncoder, name='encoder', uses_with={'device': device, 'traversal_paths': ['r']}) \
  .add(uses=SimpleIndexer, name='chunks_indexer', workspace='workspace') \
  .add(uses=SimpleRanker, name='ranker') \
  .add(uses=LMDBStorage, workspace='workspace', name='root_indexer', uses_with={'default_traversal_paths': ['m']})

Let's plot our Flow

In [ ]:
query_flow.plot()

We create the following helper function in order to plot the result documents:

In [ ]:
import matplotlib.pyplot as plt

def show_docs(docs):
    for doc in docs[:3]:
        plt.imshow(doc.blob)
        plt.show()

Finally, we can start querying. We will use images inside the query folder.
For each image, we will create a Jina Document. Then we send our documents to the query Flow and receive the response. 

For each query document, we can print the image and it's top 3 search results

In [ ]:
import glob
with query_flow:
    docs = [Document(uri=filename) for filename in glob.glob('query/*.jpg')]
    for doc in docs:
        doc.convert_image_uri_to_blob()
    resp = query_flow.post('/search', docs, return_results=True)
for doc in resp[0].docs:
    print('query:')
    plt.imshow(doc.blob)
    plt.show()
    print('results:')
    show_docs(doc.matches)

## License

Copyright 2020-2021 Jina AI Limited.  All rights reserved.


                                 Apache License
                           Version 2.0, January 2004
                        http://www.apache.org/licenses/

   TERMS AND CONDITIONS FOR USE, REPRODUCTION, AND DISTRIBUTION

   1. Definitions.

      "License" shall mean the terms and conditions for use, reproduction,
      and distribution as defined by Sections 1 through 9 of this document.

      "Licensor" shall mean the copyright owner or entity authorized by
      the copyright owner that is granting the License.

      "Legal Entity" shall mean the union of the acting entity and all
      other entities that control, are controlled by, or are under common
      control with that entity. For the purposes of this definition,
      "control" means (i) the power, direct or indirect, to cause the
      direction or management of such entity, whether by contract or
      otherwise, or (ii) ownership of fifty percent (50%) or more of the
      outstanding shares, or (iii) beneficial ownership of such entity.

      "You" (or "Your") shall mean an individual or Legal Entity
      exercising permissions granted by this License.

      "Source" form shall mean the preferred form for making modifications,
      including but not limited to software source code, documentation
      source, and configuration files.

      "Object" form shall mean any form resulting from mechanical
      transformation or translation of a Source form, including but
      not limited to compiled object code, generated documentation,
      and conversions to other media types.

      "Work" shall mean the work of authorship, whether in Source or
      Object form, made available under the License, as indicated by a
      copyright notice that is included in or attached to the work
      (an example is provided in the Appendix below).

      "Derivative Works" shall mean any work, whether in Source or Object
      form, that is based on (or derived from) the Work and for which the
      editorial revisions, annotations, elaborations, or other modifications
      represent, as a whole, an original work of authorship. For the purposes
      of this License, Derivative Works shall not include works that remain
      separable from, or merely link (or bind by name) to the interfaces of,
      the Work and Derivative Works thereof.

      "Contribution" shall mean any work of authorship, including
      the original version of the Work and any modifications or additions
      to that Work or Derivative Works thereof, that is intentionally
      submitted to Licensor for inclusion in the Work by the copyright owner
      or by an individual or Legal Entity authorized to submit on behalf of
      the copyright owner. For the purposes of this definition, "submitted"
      means any form of electronic, verbal, or written communication sent
      to the Licensor or its representatives, including but not limited to
      communication on electronic mailing lists, source code control systems,
      and issue tracking systems that are managed by, or on behalf of, the
      Licensor for the purpose of discussing and improving the Work, but
      excluding communication that is conspicuously marked or otherwise
      designated in writing by the copyright owner as "Not a Contribution."

      "Contributor" shall mean Licensor and any individual or Legal Entity
      on behalf of whom a Contribution has been received by Licensor and
      subsequently incorporated within the Work.

   2. Grant of Copyright License. Subject to the terms and conditions of
      this License, each Contributor hereby grants to You a perpetual,
      worldwide, non-exclusive, no-charge, royalty-free, irrevocable
      copyright license to reproduce, prepare Derivative Works of,
      publicly display, publicly perform, sublicense, and distribute the
      Work and such Derivative Works in Source or Object form.

   3. Grant of Patent License. Subject to the terms and conditions of
      this License, each Contributor hereby grants to You a perpetual,
      worldwide, non-exclusive, no-charge, royalty-free, irrevocable
      (except as stated in this section) patent license to make, have made,
      use, offer to sell, sell, import, and otherwise transfer the Work,
      where such license applies only to those patent claims licensable
      by such Contributor that are necessarily infringed by their
      Contribution(s) alone or by combination of their Contribution(s)
      with the Work to which such Contribution(s) was submitted. If You
      institute patent litigation against any entity (including a
      cross-claim or counterclaim in a lawsuit) alleging that the Work
      or a Contribution incorporated within the Work constitutes direct
      or contributory patent infringement, then any patent licenses
      granted to You under this License for that Work shall terminate
      as of the date such litigation is filed.

   4. Redistribution. You may reproduce and distribute copies of the
      Work or Derivative Works thereof in any medium, with or without
      modifications, and in Source or Object form, provided that You
      meet the following conditions:

      (a) You must give any other recipients of the Work or
          Derivative Works a copy of this License; and

      (b) You must cause any modified files to carry prominent notices
          stating that You changed the files; and

      (c) You must retain, in the Source form of any Derivative Works
          that You distribute, all copyright, patent, trademark, and
          attribution notices from the Source form of the Work,
          excluding those notices that do not pertain to any part of
          the Derivative Works; and

      (d) If the Work includes a "NOTICE" text file as part of its
          distribution, then any Derivative Works that You distribute must
          include a readable copy of the attribution notices contained
          within such NOTICE file, excluding those notices that do not
          pertain to any part of the Derivative Works, in at least one
          of the following places: within a NOTICE text file distributed
          as part of the Derivative Works; within the Source form or
          documentation, if provided along with the Derivative Works; or,
          within a display generated by the Derivative Works, if and
          wherever such third-party notices normally appear. The contents
          of the NOTICE file are for informational purposes only and
          do not modify the License. You may add Your own attribution
          notices within Derivative Works that You distribute, alongside
          or as an addendum to the NOTICE text from the Work, provided
          that such additional attribution notices cannot be construed
          as modifying the License.

      You may add Your own copyright statement to Your modifications and
      may provide additional or different license terms and conditions
      for use, reproduction, or distribution of Your modifications, or
      for any such Derivative Works as a whole, provided Your use,
      reproduction, and distribution of the Work otherwise complies with
      the conditions stated in this License.

   5. Submission of Contributions. Unless You explicitly state otherwise,
      any Contribution intentionally submitted for inclusion in the Work
      by You to the Licensor shall be under the terms and conditions of
      this License, without any additional terms or conditions.
      Notwithstanding the above, nothing herein shall supersede or modify
      the terms of any separate license agreement you may have executed
      with Licensor regarding such Contributions.

   6. Trademarks. This License does not grant permission to use the trade
      names, trademarks, service marks, or product names of the Licensor,
      except as required for reasonable and customary use in describing the
      origin of the Work and reproducing the content of the NOTICE file.

   7. Disclaimer of Warranty. Unless required by applicable law or
      agreed to in writing, Licensor provides the Work (and each
      Contributor provides its Contributions) on an "AS IS" BASIS,
      WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or
      implied, including, without limitation, any warranties or conditions
      of TITLE, NON-INFRINGEMENT, MERCHANTABILITY, or FITNESS FOR A
      PARTICULAR PURPOSE. You are solely responsible for determining the
      appropriateness of using or redistributing the Work and assume any
      risks associated with Your exercise of permissions under this License.

   8. Limitation of Liability. In no event and under no legal theory,
      whether in tort (including negligence), contract, or otherwise,
      unless required by applicable law (such as deliberate and grossly
      negligent acts) or agreed to in writing, shall any Contributor be
      liable to You for damages, including any direct, indirect, special,
      incidental, or consequential damages of any character arising as a
      result of this License or out of the use or inability to use the
      Work (including but not limited to damages for loss of goodwill,
      work stoppage, computer failure or malfunction, or any and all
      other commercial damages or losses), even if such Contributor
      has been advised of the possibility of such damages.

   9. Accepting Warranty or Additional Liability. While redistributing
      the Work or Derivative Works thereof, You may choose to offer,
      and charge a fee for, acceptance of support, warranty, indemnity,
      or other liability obligations and/or rights consistent with this
      License. However, in accepting such obligations, You may act only
      on Your own behalf and on Your sole responsibility, not on behalf
      of any other Contributor, and only if You agree to indemnify,
      defend, and hold each Contributor harmless for any liability
      incurred by, or claims asserted against, such Contributor by reason
      of your accepting any such warranty or additional liability.

   END OF TERMS AND CONDITIONS

   Copyright 2020-2021 Jina AI Limited

   Licensed under the Apache License, Version 2.0 (the "License");
   you may not use this file except in compliance with the License.
   You may obtain a copy of the License at

       http://www.apache.org/licenses/LICENSE-2.0

   Unless required by applicable law or agreed to in writing, software
   distributed under the License is distributed on an "AS IS" BASIS,
   WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
   See the License for the specific language governing permissions and
   limitations under the License.

## Source
[Jina docs - Find Small Images Inside Large Images](https://docs.jina.ai/datatype/image/small-images-inside-big-images/)